# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [47]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, WebsiteSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
website_tool = WebsiteSearchTool()

## Creating Agents

In [48]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions. "
              "If you are not able get data from any website try different source."
    "Always try to use diverse data source. ",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool, website_tool]
)

In [36]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options."
    "If you are not able get data from any website try different source."
    "Always try to use diverse data source. ",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [ ]:
data_fetcher_agent = Agent(
)

In [37]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy."
    "If you are not able get data from any website try different source."
    "Always try to use diverse data source. ",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [38]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance."
    "If you are not able get data from any website try different source."
    "Always try to use diverse data source. ",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [39]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [40]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [41]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [42]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [43]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2025-08-24 13:35:43,200 - 139821081398144 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [44]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'DHER',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [45]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (DHER). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate tasks to the appropriate team members and ask the right questions to gather all necessary information to analyze market data for DHER.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for DHER", "context": "Use statistical modeling and machine learning to identify trends and predict market movements for DHER."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for DHER to identify trends and predict market movements. I should use the tools available to get the information needed for analysis.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/i

Thought: I have gathered information on DHER market analysis strategies from various sources.

Final Answer: 
1. Title: DHER Technical Analysis, RSI and Moving Averages - Investing.com
Link: https://www.investing.com/equities/delivery-hero-ag-technical
Snippet: Track the 200-day and 50-day moving averages to assess long-term and short-term trends, while identifying key buy or sell signals.

2. Title: DHER technical analysis - delivery hero se - TradingView
Link: https://www.tradingview.com/symbols/XETR-DHER/technicals/
Snippet: Traders use it to see if the asset is oversold or overbought. We've counted how many oscillators show the neutral, sell, and buy trends for DELIVERY HERO SE.

3. Title: DELIVERY HERO SE (DHER.DE) Stock Technical Analysis
Link: https://www.chartmill.com/stock/quote/DHER.DE/technical-analysis
Snippet: DHER is currently trading in the middle of its 52-week range. The S&P500 Index, however, is currently trading near new highs, so DHER is lagging the market.

4. Titl

I need to gather insights on market opportunities and threats for DHER to develop and refine trading strategies accordingly.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/dher?mod=quote_search"} 

marketwatch.comPlease enable JS and disable any ad blocker

Action: Search the internet
Action Input: {"search_query": "Delivery Hero SE stock market insights"} 


Search results: Title: DHER-DE: Delivery Hero SE - Stock Price, Quote and News - CNBC
Link: https://www.cnbc.com/quotes/DHER-DE
Snippet: Key Stats
---
Title: Delivery Hero SE (DHER.DE) Stock Price, News, Quote & History
Link: https://finance.yahoo.com/quote/DHER.DE/
Snippet: Find the latest Delivery Hero SE (DHER.DE) stock quote, history, news and other vital information to help you with your stock trading and investing.
---
Title: Delivery Hero AG Stock Price Today | ETR: DHER Live - Investing.com
Link: https://www.investing.com/equities/delivery-hero-ag
Snippet: View toda

Final Answer: 
What Is a Trading Strategy? A trading strategy is a systematic methodology used for buying and selling in the securities markets. A trading strategy is based on predefined rules and criteria used when making trading decisions. A trading strategy may be simple or complex, and involve considerations such as investment style (e.g., value vs. growth), market cap, technical indicators, fundamental analysis, industry sector, level of portfolio diversification, time horizon or holding period, risk tolerance, leverage, tax considerations, and so on. The key is that a trading strategy be set using objective data and analysis and is adhered to diligently. At the same time, a trading strategy should be periodically re-evaluated and tweaked as market conditions or individual goals change.

> Finished chain.
 

What Is a Trading Strategy? A trading strategy is a systematic methodology used for buying and selling in the securities markets. A trading strategy is based on predefined rul

I need to delegate the task of analyzing potential risks associated with the proposed trading strategies and execution plans for DHER to the Risk Advisor. They are best equipped to identify and assess risks, as well as recommend mitigation strategies.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Analyze potential risks associated with the proposed trading strategies and execution plans for DHER, and suggest mitigation strategies", "context": "The detailed execution plans for DHER involve a multi-faceted approach, with the Trading Strategy Developer and Data Analyst providing insights and strategies for executing trades. The Risk Advisor should thoroughly analyze all potential risks and provide comprehensive mitigation recommendations."}


> Entering new CrewAgentExecutor chain...
I need to gather information on DHER's proposed trading strategies and execution plans to fully analyze the potential risks and provide mitigation strategies.

Action

- Display the final result as Markdown.

In [46]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for DHER:

Introduction:
The proposed trading strategies and execution plans for DHER involve a comprehensive approach that integrates insights from the Trading Strategy Developer and Data Analyst. The focus is on analyzing market data, tracking moving averages, assessing trends, identifying buy/sell signals, utilizing technical analysis tools, and leveraging statistical modeling and machine learning techniques to optimize trading decisions. This report aims to identify potential risks associated with these strategies and provide recommendations for mitigating these risks effectively.

Risk Analysis:
1. Market Volatility:
One of the significant risks associated with trading DHER is market volatility. Given the dynamic nature of the securities markets, sudden price fluctuations can impact trading outcomes significantly. To mitigate this risk, it is essential to implement stop-loss orders, closely monitor market trends, and diversify trading strategies to reduce exposure to individual stocks.

2. Liquidity Risk:
Another potential risk is liquidity risk, where trading volumes may not be sufficient to execute trades at desired prices. This can lead to slippage and affect overall trading performance. To address this risk, it is crucial to analyze trading volumes, implement limit orders, and consider trading during high-volume periods to enhance liquidity.

3. Regulatory Risk:
Trading activities are subject to regulatory oversight, and changes in regulations can impact trading strategies and execution plans. To mitigate regulatory risk, it is essential to stay updated on regulatory changes, comply with reporting requirements, and engage legal counsel to ensure trading activities adhere to applicable laws and regulations.

4. Counterparty Risk:
Engaging with third-party entities, such as brokerages and trading platforms, introduces counterparty risk. In the event of a counterparty default, trading positions may be at risk. To mitigate this risk, it is crucial to conduct due diligence on counterparties, monitor financial health, and establish contingency plans in case of default.

Mitigation Strategies:
1. Risk Management Framework:
Develop a comprehensive risk management framework that outlines risk identification, assessment, mitigation strategies, and monitoring processes. Implement risk controls, such as position limits, stop-loss orders, and portfolio diversification to manage risks effectively.

2. Continuous Monitoring:
Establish a robust monitoring system that tracks market indicators, trading performance, and risk exposure in real-time. Utilize risk management tools and analytics to identify emerging risks promptly and take proactive measures to mitigate potential losses.

3. Scenario Analysis:
Conduct scenario analysis to assess the impact of adverse market conditions on trading strategies. Identify worst-case scenarios, stress test trading positions, and develop contingency plans to mitigate risks effectively under different market scenarios.

Conclusion:
In conclusion, the proposed trading strategies and execution plans for DHER present various risks that require careful consideration and proactive risk management measures. By analyzing market trends, implementing risk controls, and staying informed about regulatory changes, it is possible to mitigate risks effectively and optimize trading performance. Continuous monitoring, scenario analysis, and a robust risk management framework are essential components of a successful risk mitigation strategy in the dynamic securities markets.